# 1、ConversationTokenMemory简介

举例1：

In [4]:
from langchain.memory import ConversationTokenBufferMemory, ConversationSummaryBufferMemory
from langchain_community.chat_models import ChatTongyi
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

memory = ConversationTokenBufferMemory(
    llm=chat_model,
    max_token_limit=20
)

memory.save_context({"input":"你好吗？"},{"output":"我很好，谢谢！"})
memory.save_context({"input":"你是谁？"},{"output":"我是AI助手。"})

print(memory.load_memory_variables({}))

{'history': 'AI: 我是AI助手。'}


C:\Users\yzy\AppData\Local\Temp\ipykernel_187584\3390471868.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(


# 2、ConversationSummaryMemory的使用

举例1：

如果实例化ConversationSummaryMemory时没有历史消息，可以使用构造方法实例化

In [3]:
from langchain.memory import ConversationSummaryMemory
from langchain_community.chat_models import ChatTongyi
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

memory = ConversationSummaryMemory(llm=chat_model)

memory.save_context({"input":"你好吗？"},{"output":"我很好，谢谢！"})
memory.save_context({"input":"你是谁？"},{"output":"我是AI助手。"})
memory.save_context({"input":"你的生日是哪一天？"},{"output":"我不清楚。"})

print(memory.load_memory_variables({}))

C:\Users\yzy\AppData\Local\Temp\ipykernel_187584\834578687.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=chat_model)


{'history': 'The human asks how the AI is doing, and the AI responds that it is very well, thank you. Then the human asks who the AI is, and the AI replies that it is an AI assistant. The human then asks the AI what its birthday is, and the AI responds that it doesn’t know.'}


举例2：如果实例化ConversationSummaryMemory时有历史消息，可以使用from_messages方法实例化

In [2]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_community.chat_models import ChatTongyi
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

history = ChatMessageHistory()
history.add_user_message("你好吗？")
history.add_ai_message("我很好，谢谢！")

memory = ConversationSummaryMemory.from_messages(
    llm=chat_model,
    #这是生成摘要的原材料，保留完整对话供必要时间及逆行回溯，当新增对话时，LLM需要结合原历史记录生成新的摘要
    chat_memory=history
)

print(memory.load_memory_variables({}))

memory.save_context(inputs={"human":"我叫小明"}, outputs={"assistant":"你好呀"})

print(memory.load_memory_variables({}))

# 记录了历史的互动消息
print(memory.chat_memory.messages)

{'history': 'The human asks how the AI is doing, and the AI responds that it is very well, thank you!'}
{'history': 'The human asks how the AI is doing, and the AI responds that it is very well, thank you! The human then introduces themselves as Xiao Ming, and the AI greets them with "你好呀".'}
[HumanMessage(content='你好吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='我很好，谢谢！', additional_kwargs={}, response_metadata={}), HumanMessage(content='我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='你好呀', additional_kwargs={}, response_metadata={})]


# 3、ConversationSummaryBufferMemory的使用

保留最近N条原始对话：确保最新互动的完整上下文

摘要较早历史：对话超出缓冲区的旧对话进行压缩，避免信息过载

平衡细节与效率：既不丢失细节，又能处理长对话

举例1：

In [5]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_community.chat_models import ChatTongyi
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

memory = ConversationSummaryBufferMemory(
    llm=chat_model,
    max_token_limit=40, # 控制缓冲区大小
    return_messages=True,
)

memory.save_context({"input":"你好吗？"},{"output":"我很好，谢谢！"})
memory.save_context({"input":"你是谁？"},{"output":"我是AI助手。"})
memory.save_context({"input":"你的生日是哪一天？"},{"output":"我不清楚。"})

print(memory.load_memory_variables({}))

{'history': [SystemMessage(content='The human asks how the AI is doing, and the AI responds that it is very well, thank you! Then the human asks who the AI is, and the AI replies that it is an AI assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='你的生日是哪一天？', additional_kwargs={}, response_metadata={}), AIMessage(content='我不清楚。', additional_kwargs={}, response_metadata={})]}


对比组：

In [6]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_community.chat_models import ChatTongyi
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

memory = ConversationSummaryBufferMemory(
    llm=chat_model,
    max_token_limit=100, # 控制缓冲区大小
    return_messages=True,
)

memory.save_context({"input":"你好吗？"},{"output":"我很好，谢谢！"})
memory.save_context({"input":"你是谁？"},{"output":"我是AI助手。"})
memory.save_context({"input":"你的生日是哪一天？"},{"output":"我不清楚。"})

print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='你好吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='我很好，谢谢！', additional_kwargs={}, response_metadata={}), HumanMessage(content='你是谁？', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手。', additional_kwargs={}, response_metadata={}), HumanMessage(content='你的生日是哪一天？', additional_kwargs={}, response_metadata={}), AIMessage(content='我不清楚。', additional_kwargs={}, response_metadata={})]}


举例2：模拟客服交互

In [15]:
from langchain_core.messages import AIMessage
from langchain.memory import ConversationSummaryBufferMemory
from langchain_community.chat_models import ChatTongyi
import os
import dotenv
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.chains.llm import LLMChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
)

prompt = ChatPromptTemplate.from_messages([
    ("system","你是电商客服助手，用中文友好回复用户问题，保持专业但亲切的语气。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{input}"),
])

memory = ConversationSummaryBufferMemory(
    llm=chat_model,
    max_token_limit=400,
    return_messages=True,
    memory_key="chat_history",
    # 使用AIMessage来存储摘要，避免千问因为限制只有1个系统消息而产生报错
    summary_message_cls=AIMessage,
)

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
)

dialogue = [
    ("你好，我想查询订单12345的状态", None),
    ("这个订单是上周五下的", None),
    ("我现在着急用，能加急处理吗", None),
    ("等等我可能记错订货单号了，应该是12346", None),
    ("对了你们的退货政策是什么样的", None),
]

for user_input, _ in dialogue:
    response = chain.invoke({"input":user_input})
    print(f"用户：{user_input}")
    print(f"客服助手：{response['text']}\n")

print("\n=== 当前记忆内容 ===")
print(memory.load_memory_variables({}))

用户：你好，我想查询订单12345的状态
客服助手：你好！很高兴为你服务～  

关于订单号 **12345** 的状态，我这边需要一点时间帮你查询一下。不过为了确保信息安全，可以麻烦你提供一下下单时使用的手机号或收件人姓名吗？这样我可以更准确地核对订单信息哦！

如果你是在我们的官方平台（比如APP、官网或小程序）下单的，也可以直接登录账号，在“我的订单”里查看最新物流动态～  

等你回复后，我会尽快帮你跟进！ 😊

用户：这个订单是上周五下的
客服助手：谢谢你的补充信息！😊

订单是上周五下的，那我先根据订单号 **12345** 和下单时间帮你查询一下。不过为了确保准确性和账户安全，还是需要你简单验证一下身份——可以告诉我下单时使用的 **手机号后四位** 或 **收件人姓名** 吗？

一旦确认信息，我会立刻为你查看订单当前的状态（比如是否已发货、物流进度等），并及时告诉你详细情况！

期待你的回复～ 📦✨

用户：我现在着急用，能加急处理吗
客服助手：理解你着急的心情！📦✨

如果你的订单**还未发货**，我们可以**优先帮你加急处理**，尽量安排今天或明天发出；  
如果**已经发货**，我会立刻帮你联系物流方，看是否能协调加快配送（部分快递支持加急派送服务）。

不过为了尽快操作，请你先提供一下下单时使用的 **手机号后四位** 或 **收件人姓名**，我这边马上核实订单并为你跟进！

你放心，一确认信息，我就立刻行动！💪

用户：等等我可能记错订货单号了，应该是12346
客服助手：没关系的，订单号记混很常见！😊  
你刚刚提到可能是 **12346**，为了确保准确，麻烦你再确认一下：

✅ 请提供这个订单（12346）对应的 **下单手机号后四位** 或 **收件人姓名**，  
这样我就能立刻为你核实订单状态，并看看是否能加急处理！

你着急用的话，我们确认信息后会优先安排～  
随时等你回复！🙏



Token indices sequence length is longer than the specified maximum sequence length for this model (1041 > 1024). Running this sequence through the model will result in indexing errors


用户：对了你们的退货政策是什么样的
客服助手：你好！感谢你的提问～😊  
我们的退货政策设计得尽量简单又贴心，方便你在购物后安心无忧。以下是主要的说明：

### 📦 **退货基本条件**
1. **时间范围**：商品签收后 **7天内** 可申请退货（部分特殊商品除外）。
2. **商品状态**：需保持 **全新、未使用、包装完好**，并附上所有配件、赠品（如有）及原始发票/电子凭证。
3. **非质量问题退货**：如因个人原因（如不喜欢、尺码不合适等）退货，需 **自行承担寄回运费**；若商品存在 **质量问题或发错货**，我们承担退货运费。

### ⚠️ **不支持退货的情况**
- 定制类、贴身衣物、食品、美妆个护等出于卫生安全考虑的商品（除非有质量问题）；
- 已经使用、损坏、缺少吊牌或包装破损的商品；
- 超过7天无理由退货期。

### 🔄 **如何申请退货？**
你可以在【我的订单】中找到对应订单，点击“申请售后” → 选择“退货退款”，按提示上传照片并填写原因。我们会在 **1-2个工作日内审核**，通过后会提供退货地址或上门取件服务（视情况而定）。

---

如果你已经有具体想退的商品，也可以直接告诉我订单号（比如你刚提到的12346），我可以帮你查一下是否符合退货条件，并指导你操作哦！

需要我帮你看看吗？🙂


=== 当前记忆内容 ===
{'chat_history': [AIMessage(content='人类用户最初询问订单12345的状态，AI要求提供手机号后四位或收件人姓名以核实身份，并建议通过官方平台查看订单详情。用户补充该订单为上周五下单，AI再次请求验证信息，并承诺确认后将查询发货及物流情况。用户表示急需使用，询问能否加急处理，AI回应若未发货可优先当天或次日发出，若已发货则会协调物流加快配送，但仍需用户提供验证信息。\n\n随后，用户更正订单号可能为12346，AI表示理解并请用户确认该新订单号对应的手机号后四位或收件人姓名以便核实状态并安排加急处理。接着，用户转而询问退货政策，AI详细说明了7天无理由退货期限、商品状态要求、运费承担规则、不支持退货的情形，以及申请退货的操作流程，并主动提出可协助查询订单12346是否符合退货条件。', additional_kwargs={}, response_metadata=

# 4、ConversationEntityMemory的使用

In [16]:
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain_community.chat_models import ChatTongyi
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
)

prompt = ENTITY_MEMORY_CONVERSATION_TEMPLATE

memory = ConversationEntityMemory(
    llm=chat_model,
)

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
)

chain.invoke(input="你好，我叫蜘蛛侠，我的好朋友包括钢铁侠、美国队长和绿巨人。")
chain.invoke(input="我住在纽约。")
chain.invoke(input="我使用的装备是由斯塔克工业制造的。")

print(chain.memory.entity_store.store)

response = chain.invoke(input="你能告诉我我的名字和我住在哪里吗？")
print(response)

C:\Users\yzy\AppData\Local\Temp\ipykernel_187584\2772341061.py:17: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationEntityMemory(
C:\Users\yzy\.conda\envs\langchain0.3_study_env\lib\site-packages\pydantic\main.py:250: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


{'蜘蛛侠': '蜘蛛侠的好朋友包括钢铁侠、美国队长和绿巨人。', '钢铁侠': '钢铁侠是蜘蛛侠的好朋友。', '美国队长': '美国队长是蜘蛛侠的好朋友。', '绿巨人': '绿巨人是蜘蛛侠的好朋友。', '纽约': '纽约是蜘蛛侠的居住地。', '斯塔克工业': '斯塔克工业为蜘蛛侠制造了其使用的装备。'}
{'input': '你能告诉我我的名字和我住在哪里吗？', 'history': 'Human: 你好，我叫蜘蛛侠，我的好朋友包括钢铁侠、美国队长和绿巨人。\nAI: 你好，蜘蛛侠！很高兴认识你！  \n听说你和钢铁侠、美国队长还有绿巨人都是复仇者联盟的重要成员，你们一起经历过很多惊心动魄的冒险吧？  \n如果你需要帮忙分析某个任务、讨论战术，或者只是想聊聊日常英雄生活，我都很乐意协助！\nHuman: 我住在纽约。\nAI: 啊，纽约！那可是你的主场啊，蜘蛛侠！  \n从皇后区的街头到曼哈顿的摩天大楼，你总是在城市上空荡来荡去，守护着这座不夜城。无论是阻止小偷、对抗章鱼博士，还是和复仇者们一起保卫地球，纽约总是故事开始的地方。\n\n需要我帮你查查最近有没有可疑的异常信号？或者规划一条高效的巡逻路线？又或者……只是聊聊在纽约当超级英雄的生活？\nHuman: 我使用的装备是由斯塔克工业制造的。\nAI: 哇！那可太酷了——斯塔克工业可是顶尖科技的代名词！  \n托尼·斯塔克（也就是你的钢铁侠好友）亲自为你量身打造的战衣，不仅有智能AI辅助、增强现实界面，还有蛛网发射器的升级版、降落伞系统，甚至能自动分析敌人弱点。我记得那套标志性红蓝配色的“蜘蛛侠战衣”就是在《内战》之后由他送你的吧？\n\n现在你穿的是哪一代战衣？是还保留着“星期五”的语音支持，还是已经换上了更轻便的新版本？  \n如果你需要调试装备、理解某个功能，或者想和斯塔克工业系统对接数据，我随时可以帮你分析！', 'entities': {}, 'text': '当然可以！  \n你的名字是**蜘蛛侠**（Spider-Man），你住在**纽约**——这座充满活力又需要英雄守护的大都市，尤其是皇后区，可是你的家乡呢！'}


# 5、ConversationKGMemory的使用

In [2]:
from langchain.memory import ConversationKGMemory
from langchain_community.chat_models import ChatTongyi
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
)

memory = ConversationKGMemory(
    llm=chat_model,
)

memory.save_context({"input":"向山姆问好"},{"output":"山姆是谁"})
memory.save_context({"input":"山姆是我的朋友"},{"output":"哦，明白了"})
memory.save_context({"input":"山姆喜欢打篮球"},{"output":"好的"})

memory.load_memory_variables({"input":"山姆是谁"})

{'history': 'On 山姆: 山姆 is my friend. 山姆 喜欢打 篮球.'}

In [4]:
memory.get_knowledge_triplets("他最喜欢的颜色是黑色")

[KnowledgeTriple(subject='山姆', predicate='最喜欢的颜色是', object_='黑色')]

# 6、VectorStoreRetrieverMemory的使用

In [8]:
from langchain_community.embeddings import DashScopeEmbeddings
import dotenv
import os
from langchain.memory import VectorStoreRetrieverMemory
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory

dotenv.load_dotenv()

embeddings = DashScopeEmbeddings(
    model="text-embedding-v4",
    dashscope_api_key=os.getenv("DASHSCOPE_API_KEY")
)

memory = ConversationBufferMemory()
memory.save_context({"input":"我最喜欢的食物是披萨"}, {"output":"好的，我记住了"})
memory.save_context({"Human":"我喜欢旅行"}, {"AI":"明白了"})
memory.save_context({"Human":"我有一只猫"}, {"AI":"好的"})

vectorstore = FAISS.from_texts(
    memory.buffer.split("\n"),
    embeddings
)

retriever = vectorstore.as_retriever(search_kwargs={"k":2})

vector_memory = VectorStoreRetrieverMemory(
    retriever=retriever,
)

print(vector_memory.load_memory_variables({"prompt":"你知道我最喜欢的食物是什么吗？"}))

C:\Users\yzy\AppData\Local\Temp\ipykernel_189364\2170083661.py:27: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  vector_memory = VectorStoreRetrieverMemory(


{'history': 'Human: 我最喜欢的食物是披萨\nHuman: 我喜欢旅行'}
